In [ ]:
!pip install networkx

In [ ]:
!pip install scipy 

In [ ]:
!pip install matplotlib

In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt
from collections import deque
import concurrent.futures
import numpy as np
from scipy.stats import uniform, beta

n_nodes=1000
G = nx.erdos_renyi_graph(n_nodes, 0.02,directed=True)

for node in G.nodes():
    G.nodes[node]['weight'] = random.uniform(0, 1)

for u, v in G.edges():
    G.edges[u, v]['weights'] = (
        random.uniform(0, 1),
        random.uniform(0, 1),
        random.uniform(0, 1)
    )
threshold_dict = {v: beta(1, int(np.random.randint(1, 10))) for v in range(n_nodes)}

In [26]:
#In ra số cạnh của G
print('Số cạnh: ',G.number_of_edges())
print('Các đỉnh cạnh 1:')
for neighbor in G.neighbors(0):
    print(neighbor)

Số cạnh:  20073
Các đỉnh cạnh 1:
21
68
114
127
616
682
787
790
801
805
986


## Hàm tính ảnh hưởng

In [36]:
def gtlm_influence_spread(G, seed_set, thresholds):
    """
    G: Đồ thị với trọng số cạnh là xác suất ảnh hưởng giữa các nút.
    seed_set: List các nút ban đầu bị ảnh hưởng. Mỗi phần tử là (node, topic).
    thresholds: Dictionary chứa ngưỡng ảnh hưởng cho mỗi nút.
    
    Trả về: dict {topic: set(node bị ảnh hưởng theo topic)}
    """
    activated = dict()  # node -> topic bị ảnh hưởng đầu tiên
    influence = dict()  # node -> ảnh hưởng tổng cộng từ các hàng xóm
    
    for node in G.nodes:
        influence[node] = {0: 0, 1: 0, 2: 0}  # Khởi tạo ảnh hưởng cho tất cả các chủ đề là 0
    # Khởi tạo các nút ban đầu bị ảnh hưởng
    for node, topic in seed_set:
        activated[node] = topic
        # Khởi tạo ảnh hưởng cho tất cả các topic (0, 1, 2)
        #if node not in influence:
            #influence[node] = {0: 0, 1: 0, 2: 0}
        influence[node][topic] = 1  # Đặt ảnh hưởng ban đầu là 1 cho chủ đề bị ảnh hưởng

    
    # Sử dụng hàng đợi để lan truyền ảnh hưởng
    queue = deque(seed_set)
    
    while queue:
        current_node, topic = queue.popleft()
        
        # Duyệt qua các nút hàng xóm
        for neighbor in G.neighbors(current_node):
            if neighbor in activated:
                continue  # Nếu đã bị ảnh hưởng, bỏ qua
            
            # Tính toán ảnh hưởng từ current_node đến neighbor cho topic hiện tại
            weight = G.edges[current_node, neighbor].get('weights', (0, 0, 0))
            influence_prob = weight[topic]
            
            # Cập nhật tổng ảnh hưởng cho neighbor
            influence[neighbor][topic] += influence_prob
            threshold_value = thresholds[neighbor].rvs()
            # Kiểm tra xem neighbor có bị ảnh hưởng hay không
            if influence[neighbor][topic] >= threshold_value:
                activated[neighbor] = topic
                queue.append((neighbor, topic))
    
    # Tạo kết quả phân loại các nút theo topic
    topic_influence = {0: set(), 1: set(), 2: set()}
    for node, topic in activated.items():
        topic_influence[topic].add(node)

    return topic_influence,len(activated.items())


In [37]:
def influence_spread(G, seed_set):
    """
    G: Đồ thị với trọng số cạnh là bộ 3 số tương ứng 3 chủ đề.
    seed_set: List các tuple (node, topic) – tập node ban đầu bị ảnh hưởng theo chủ đề.
    
    Trả về: dict {topic: set(node bị ảnh hưởng theo topic)}
    """
    activated = dict()  # node -> topic bị ảnh hưởng đầu tiên
    queue = deque(seed_set)

    for node, topic in seed_set:
        activated[node] = topic

    while queue:
        current_node, topic = queue.popleft()

        for neighbor in G.neighbors(current_node):
            if neighbor in activated:
                continue  # đã bị ảnh hưởng rồi

            weight_tuple = G.edges[current_node, neighbor].get('weights', (0, 0, 0))
            influence_prob = weight_tuple[topic]

            if random.random() <= influence_prob:
                activated[neighbor] = topic
                queue.append((neighbor, topic))

    topic_influence = {0: set(), 1: set(), 2: set()}
    for node, topic in activated.items():
        topic_influence[topic].add(node)

    return topic_influence

In [38]:
def monte_carlo_parallel(G, seed_set, num_simulations=100):
    from concurrent.futures import ThreadPoolExecutor

    def single_simulation(_):
        return len(set.union(*influence_spread(G, seed_set).values()))

    with ThreadPoolExecutor() as executor:
        results = list(executor.map(single_simulation, range(num_simulations)))

    return sum(results) / num_simulations


In [39]:
avg_total = monte_carlo_parallel(G, seed_set=[(0, 1)], num_simulations=1000)
print(f"Trung bình số node bị ảnh hưởng: {avg_total:.2f}")
gltm,sl = gtlm_influence_spread(G, seed_set=[(0, 1)],thresholds=threshold_dict)
print(sl)

Trung bình số node bị ảnh hưởng: 999.99
1000
